In [1]:
import cv2 
import numpy as np
import time
import math

#Loading and initialising yolov3 from opencv

In [2]:
args = {
          "confThreshold": 0.9,
          "nmsThreshold":0.4,
          "inpWidth":416,
          "inpHeight":416,
          "bboxAreaToImageArea":0.15,
          "team0Name":'MIL',
          "team1Name":'CAVS',
          "colorBoundaries":[
                        ([ 56, -7 ,186], [196, 133, 266]), #white/team0
                        ([160-70,170-80, 60-30], [160+70,170+80, 60+30]) #red/team1
                        ]
        }


In [3]:
with open("/Users/sandeep/Desktop/dataandmodles/models/teamDetection/coco.names", 'rt') as f:
    classes = f.read().rstrip('\n').split('\n')

In [4]:
# Get the names of the output layers of the CNN network
# net : an OpenCV DNN module network object
def getOutputsNames(net):
    # Get the names of all the layers in the network
    layersNames = net.getLayerNames()
    # Get the names of the output layers, i.e. the layers with unconnected outputs
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [5]:
rootDir = '/Users/sandeep/Desktop/dataandmodles/models/teamDetection'
net = cv2.dnn.readNet(rootDir+"/yolov3.weights",rootDir+"/yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)
# change to cv2.dnn.DNN_TARGET_CPU (slower) if this causes issues (should fail gracefully if OpenCL not available)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)
output_layer_names = getOutputsNames(net)

#Opencv setup

In [6]:
# dummy on trackbar callback function
def on_trackbar(val):
    return

In [7]:
windowName = 'YOLOv3 Team detection'
cv2.namedWindow(windowName , cv2.WINDOW_NORMAL)
trackbarName = 'reporting confidence > (x 0.01)'
cv2.createTrackbar(trackbarName,windowName,70,100, on_trackbar)


#HelperFunction:Drawing Prediction

In [8]:
def drawPred(image,team,class_name, confidence, left, top, right, bottom, colour):
    # Draw a bounding box.
    cv2.rectangle(image, (left, top), (right, bottom), colour, 3)

    # construct label
    label = '%s:%.2f' % (class_name, confidence)
    label = label+f'| team:{team}'

    #Display the label at the top of the bounding box
    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(image, (left, top - round(1.5*labelSize[1])),
        (left + round(1.5*labelSize[0]), top + baseLine), (255, 255, 255), cv2.FILLED)
    cv2.putText(image, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 1)

#HelperFunction:Post Procces

In [9]:
def postprocess(image, results, threshold_confidence, threshold_nms):
    frameHeight = image.shape[0]
    frameWidth = image.shape[1]

    # Scan through all the bounding boxes output from the network and..
    # 1. keep only the ones with high confidence scores.
    # 2. assign the box class label as the class with the highest score.
    # 3. construct a list of bounding boxes, class labels and confidence scores

    classIds = []
    confidences = []
    boxes = []
    for result in results:
        for detection in result:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > threshold_confidence:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences
    classIds_nms = []
    confidences_nms = []
    boxes_nms = []

    indices = cv2.dnn.NMSBoxes(boxes, confidences, threshold_confidence, threshold_nms)
    for i in indices:
        i = i[0]
        classIds_nms.append(classIds[i])
        confidences_nms.append(confidences[i])
        boxes_nms.append(boxes[i])

    # return post processed lists of classIds, confidences and bounding boxes
    return (classIds_nms, confidences_nms, boxes_nms)

#Filtering the preditction

In [10]:
def check_bbox_size(bboxW,bboxH,imgW,imgH):
    bboxToImg = (bboxW*bboxH) / (imgW * imgH)
    return bboxToImg <= args['bboxAreaToImageArea']

In [11]:
def check_label(label):
    return label == 'person'

#ROI color detection

In [12]:
def getRoi(frame, left,top,right,bottom):
    '''
    Helper function for detect_teams
    Returns ROI(region of interest) 
    '''
    roi = frame[top:bottom , left:right,:]
    return roi

In [13]:
def countNonBalckPix(roiMasked):
    '''
    Helper function for findColorRatio
    Returns the number of non black pixels in the roi
    '''
    return roiMasked.any(axis = -1).sum()

In [14]:
def getColorRatio(roi,show=False):
    '''
    Helper function for detect teams
    Returns a list, that contains percentage of the pixel that have the team %colors
    Example: [0.9 , 0.1]. 90% of the pixels are of team 1
    '''
    ratioList = []
    
    for teamColorLower,teamColorUpper in args['colorBoundaries']:
        mask = cv2.inRange(roi , np.array(teamColorLower) , np.array(teamColorUpper))
        roiMasked = cv2.bitwise_and(roi,roi,mask=mask)
        totalColorPix = countNonBalckPix(roiMasked)
        totalPix = countNonBalckPix(roi)
        colorPixRatio = totalColorPix / totalPix
        ratioList.append(colorPixRatio)
        #print(f'totalColrPix:{totalColorPix} , totalPx:{totalPix}')
        if show == True:
            cv2.imshow("images", np.hstack([roi,roiMasked]))
            if cv2.waitKey(0) & 0xFF == ord('q'):
              cv2.destroyAllWindows() 

    return np.array(ratioList)    

In [15]:
# img = cv2.imread('/Users/sandeep/Desktop/dataandmodles/data/cavs.JPG')
# roi = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
# getColorRatio(roi, show=True)

In [16]:
def compareRatio(ratioList):
    '''
    Helper function for detectTeam
    Finds the team with highest color ratio.
    Returns string team names or "Uncertain" if not sure
    '''
    maxRatio = max(ratioList)
    if maxRatio < 0.1:
        return 'Uncertain'
    else:
        print("Team1", ratioList[1])
        print("Team0", ratioList[0])        
        if ratioList[1] > ratioList[0]:
            return args['team1Name']
        elif ratioList[1] <= ratioList[0]:
            return args['team0Name']
            
    

In [17]:
def detectTeam(img,left,top,right,bottom):
    '''
    Given an image(BGR) and the location of ROI
    Finds the team based on ROI color
    '''
    roi = getRoi(img,left,top,right,bottom)
    roiHSV = np.array(cv2.cvtColor(roi, cv2.COLOR_BGR2HSV))
    ratioList = getColorRatio(roiHSV)
    team = compareRatio(ratioList)
    return team

#Object detection boiler template

In [18]:
videoPath ='/Users/sandeep/Desktop/dataandmodles/data/3-Pointer2.mov'
cap = cv2.VideoCapture(videoPath)
frameCount = 0 
rawFrame=[]
while cap.isOpened():
    ret,frame = cap.read()
    frameCopy = frame[:]
    if ret:
        start_t = cv2.getTickCount()
        
        #do stuff
        # create a 4D tensor (OpenCV 'blob') from image frame (pixels scaled 0->1, image resized)
        tensor = (cv2.dnn.blobFromImage(frame , 1/255 , (args["inpWidth"], args["inpHeight"]) , [0,0,0] , 1, 
                                        crop=False))
        # set the input to the CNN network
        net.setInput(tensor)
        results = net.forward(output_layer_names)
        
        args['confThreshold'] = cv2.getTrackbarPos(trackbarName,windowName) / 100
        classIDs, confidences, boxes = (postprocess(frame, results, args["confThreshold"], 
                                                    args["nmsThreshold"]))
        for detected_object in range(0, len(boxes)):
            
            box = boxes[detected_object]
            left = box[0]
            top = box[1]
            width = box[2]
            height = box[3]
    
            bboxFit = check_bbox_size(width,height, *frame.shape[0:-1])
            labelFit = check_label(classes[classIDs[detected_object]])
            if bboxFit and labelFit and left>0:
                team= detectTeam(frameCopy, left ,top, left+width,top+height)
                #team = 'NULL'
                getTeamColor
                (drawPred(frame,team,classes[classIDs[detected_object]], 
                          confidences[detected_object], 
                          left, top, left + width, top + height, 
                          (255, 178, 50)))

            t,_ = net.getPerfProfile()
            inference_t = (t * 1000.0 / cv2.getTickFrequency())
            label = ('Inference time: %.2f ms' % inference_t) + (' (Framerate: %.2f fps' % (1000 / inference_t)) + ')'
            cv2.putText(frame, label, (0, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255))
            if frameCount == 496: 
                print(f'{left},{top},{left + width},{top + height}')
                rawFrame = frame[:,:,:]
            frameCount += 1
#         End of do stuff
        
        cv2.imshow(windowName,frame)
        (cv2.setWindowProperty(windowName, cv2.WND_PROP_FULLSCREEN,
                                cv2.WINDOW_FULLSCREEN&False))    
             
        time_now = cv2.getTickCount()
        stop_t = ((time_now - start_t)/cv2.getTickFrequency())*1000
    
        #cv2.imshow("YOLO" , frame)
        
        key = cv2.waitKey(max(2, 40 - int(math.ceil(stop_t)))) & 0xFF
        if key == ord('q'):
            break  
    else:
        cap.release()
        break
cap.release()
cv2.destroyAllWindows()
    

Team1 0.14227551766913707
Team0 0.004643913610483794
Team1 0.26681426814268144
Team0 0.02666666666666667
Team1 0.03444264406497985
Team0 0.15571897461464954
Team1 0.2842982178635453
Team0 0.052479876269816164
Team1 0.058993825884021155
Team0 0.23597530353608462
Team1 0.11779591836734694
Team0 0.16661224489795917
Team1 0.20167238421955402
Team0 0.03486277873070326
Team1 0.252088205813565
Team0 0.022719679251587035
Team1 0.13997531185031184
Team0 0.006561850311850312
Team1 0.249835076807087
Team0 0.038686268966167184
Team1 0.038486627527723416
Team0 0.16475048923679061
Team1 0.2571563088512241
Team0 0.0509730069052103
Team1 0.10634254493625782
Team0 0.16667986380552696
Team1 0.08003140855279053
Team0 0.2259603769026335
Team1 0.20220679394991323
Team0 0.03587073311843954
Team1 0.2522552622786502
Team0 0.022719679251587035
Team1 0.2043201502660962
Team0 0.039723120804201886
Team1 0.14088001044522783
Team0 0.008258258258258258
Team1 0.03241045923972753
Team0 0.15381234893430015
Team1 0.1195

Team1 0.16427426660163055
Team0 0.021078670475925022
Team1 0.08271827182718272
Team0 0.16624662466246626
Team1 0.2608371671094408
Team0 0.06379187455452602
Team1 0.14051376267182836
Team0 0.002427741871985827
Team1 0.15612641541321615
Team0 0.07889133006591179
Team1 0.2146654010441386
Team0 0.04731846226862838
Team1 0.11624463201540056
Team0 0.1512661039537983
Team1 0.09474496426250813
Team0 0.15761046133853152
Team1 0.33145363408521306
Team0 0.01887531328320802
Team1 0.12817039211259126
Team0 0.26665161435990065
Team1 0.15794159544159544
Team0 0.0010683760683760685
Team1 0.5168264462809917
Team0 0.023008264462809916
Team1 0.4000294464075383
Team0 0.03931095406360424
Team1 0.18256843800322062
Team0 0.028784219001610305
Team1 0.0647218564845293
Team0 0.16100230414746544
Team1 0.1503475316342898
Team0 0.0029941186954197114
Team1 0.18277116402116403
Team0 0.07546296296296297
Team1 0.2075
Team0 0.047547169811320754
Team1 0.26794516847862787
Team0 0.0737727057870214
Team1 0.1150375939849624

Team1 0.06116972894305985
Team0 0.18510096416227034
Team1 0.05029935950988582
Team0 0.16812865497076024
Team1 0.12785749628896587
Team0 0.015009071416790368
Team1 0.06571634635781257
Team0 0.16036655211912945
Team1 0.2255976325888935
Team0 0.038655384473112314
Team1 0.1850476951271087
Team0 0.0313425120999953
Team1 0.23897707231040563
Team0 0.06672545561434451
Team1 0.11082859706850134
Team0 0.20101705055339517
Team1 0.18168409761330115
Team0 0.055913113435237326
Team1 0.23472747225948054
Team0 0.016095598097792953
Team1 0.16646466176804786
Team0 0.008976592682372548
Team1 0.397415611814346
Team0 0.016438115330520395
Team1 0.15873015873015872
Team0 0.006715506715506716
Team1 0.04809782608695652
Team0 0.17581521739130435
Team1 0.056422781494460374
Team0 0.17815298573501823
Team1 0.06276730072094823
Team0 0.16239664372123336
Team1 0.13590222373111527
Team0 0.021049057884909184
Team1 0.18317019861256295
Team0 0.025325477525420508
Team1 0.2042450941129355
Team0 0.039380590041382994
Team1 0

Team1 0.01636432350718065
Team0 0.1875283446712018
Team1 0.23091226472374013
Team0 0.027588038858530662
Team1 0.3019868258983559
Team0 0.09730627108659562
Team1 0.20338220164609053
Team0 0.026491769547325104
Team1 0.27851799906498365
Team0 0.007947639083683964
Team1 0.05002121177060434
Team0 0.1497936673223032
Team1 0.18537224092779647
Team0 0.05742611298166854
Team1 0.013196389666977903
Team0 0.278182384064737
Team1 0.2125486955495219
Team0 0.04810530043678432
Team1 0.16110777243589744
Team0 0.038661858974358976
Team1 0.097280285593499
Team0 0.16858471511109024
Team1 0.6001196172248804
Team0 0.006339712918660287
Team1 0.09501890514548743
Team0 0.2612197928653625
Team1 0.14903715758068892
Team0 0.36303227556278816
Team1 0.42837370242214534
Team0 0.04740484429065744
Team1 0.015430448399663892
Team0 0.20044305247880223
Team1 0.306140350877193
Team0 0.08971734892787524
Team1 0.21674430641821946
Team0 0.030020703933747412
Team1 0.21871632543103448
Team0 0.025693696120689655
Team1 0.0491385

Team1 0.008550216883550217
Team0 0.27652652652652654
Team1 0.2625008412409987
Team0 0.02338649976445252
Team1 0.3060446009389671
Team0 0.029636150234741785
Team1 0.2010177113328863
Team0 0.03561989665101968
Team1 0.01801678525816457
Team0 0.20803685458857873
Team1 0.20449214867819518
Team0 0.0708805406479825
Team1 0.13277940697295537
Team0 0.10728250244379277
Team1 0.3241648992576882
Team0 0.17941410392364793
Team1 0.14734724666834298
Team0 0.3473723912496857
Team1 0.23673960362011684
Team0 0.08586321457211593
Team1 0.16501432033487554
Team0 0.005948446794448116
Team1 0.39731721141129794
Team0 0.0049121481201587006
Team1 0.08296353463245225
Team0 0.25429288057109783
Team1 0.13250032539372641
Team0 0.05857087075361187
Team1 0.41152542372881357
Team0 0.0005423728813559322
Team1 0.008441362677117878
Team0 0.28554201300658943
Team1 0.24179536679536678
Team0 0.017117117117117116
Team1 0.21438172043010753
Team0 0.036211258697027196
Team1 0.30115442529697173
Team0 0.026100050192404217
Team1 0

Team1 0.0033851257574218883
Team0 0.19833451812734842
Team1 0.015889167188478395
Team0 0.20589386349405134
Team1 0.3013048635824436
Team0 0.021664481488418555
Team1 0.3204742625795257
Team0 0.08461538461538462
Team1 0.18688111888111889
Team0 0.0260979020979021
Team1 0.2512254901960784
Team0 0.0477580738177624
Team1 0.15128417564208782
Team0 0.11753659210162938
Team1 0.04231437561639976
Team0 0.20790870239045695
Team1 0.11026785714285714
Team0 0.14469246031746033
Team1 0.14012022493697887
Team0 0.04254411479542369
Team1 0.12506733264776457
Team0 0.1002399490720337
Team1 0.19826555023923445
Team0 0.02721291866028708
Team1 0.11500838457238681
Team0 0.09656232532140861
Team1 0.0008597883597883598
Team0 0.19328703703703703
Team1 0.2978586526973624
Team0 0.016665134944704837
Team1 0.026951360633095674
Team0 0.19399753630221359
Team1 0.18004082071113975
Team0 0.10967880545708454
Team1 0.30979694092827004
Team0 0.0971123417721519
Team1 0.26049910873440285
Team0 0.07614973262032086
Team1 0.1231

Team1 0.2737222312747814
Team0 0.003992178588887079
Team1 0.29120727129025886
Team0 0.02197194230389251
Team1 0.17600108225108224
Team0 0.07694128787878787
Team1 0.05263717944170539
Team0 0.23810875039903523
Team1 0.09043527257360191
Team0 0.12710710428698876
Team1 0.29782024005459434
Team0 0.016217735137088032
Team1 0.2700105318588731
Team0 0.06220379146919431


In [19]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [20]:
for lower , upper in args['colorBoundaries']:
    print(lower)
    print(upper)

[56, -7, 186]
[196, 133, 266]
[90, 90, 30]
[230, 250, 90]
